# Chapter 6. 일반선형모델

## 6.4 일반선형모델의 평가

* 정규선형모델에서 배운 것처럼 모델을 평가할 때 잔차의 체크는 필수

    - 다만 모집단 분포가 정규분포 이외의 분포가 되면 잔차의 취급이 크게 바뀌게 됨
    
* 이 절에서는 일반선형모델에서의 잔차 취급을 설명함

    - 잔차는 데이터와 모델의 괴리를 표현하는 중요한 지표
    
    - 모델의 손실을 파악하는 방법도 확인

### 6.4.1 분석 준비

In [1]:
# 수치계산에 사용되는 라이브러리
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats

# 그래프를 그리기 위한 라이브러리
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

# 통계모델을 추정하는데 사용하는 라이브러리
import statsmodels.formula.api as smf
import statsmodels.api as sm

# 표시 자릿수 지정
%precision 3

# 그래프를 주피터 노트북에 그리기 위한 설정
%matplotlib inline

In [2]:
# 데이터 읽어 들이기
test_result = pd.read_csv("py_source/6-3-1-logistic-regression.csv")

# 로지스틱 회귀모델 모델링
mod_glm = smf.glm("result ~ hours", data = test_result, family = sm.families.Binomial()).fit()

# 로지스틱 회귀모델 모델링
mod_glm_null = smf.glm("result ~ 1", data = test_result, family = sm.families.Binomial()).fit()

In [3]:
mod_glm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                 result   No. Observations:                  100
Model:                            GLM   Df Residuals:                       98
Model Family:                Binomial   Df Model:                            1
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -34.014
Date:                Sun, 18 Sep 2022   Deviance:                       68.028
Time:                        19:14:00   Pearson chi2:                     84.9
No. Iterations:                     6   Pseudo R-squ. (CS):             0.5032
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -4.5587      0.901     -5.061      0.000      -6.324      -2.793
hours          0.9289      0.174      5.345      0.000       0.588       1.270
==============================================================================
"""

In [4]:
mod_glm_null.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                 result   No. Observations:                  100
Model:                            GLM   Df Residuals:                       99
Model Family:                Binomial   Df Model:                            0
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -68.994
Date:                Sun, 18 Sep 2022   Deviance:                       137.99
Time:                        19:14:00   Pearson chi2:                     100.
No. Iterations:                     4   Pseudo R-squ. (CS):          3.331e-16
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.1603      0.201     -0.799      0.424      -0.554       0.233
==============================================================================
"""

### 6.4.2 피어슨 잔차

* 이항분포에서 **피어슨 잔차** 

    - y는 종속변수(이항확률 변수, 이 경우에는 합격여부 데이터)
    
    - N은 시행횟수, $\hat{p}$은 추측한 성공확률(즉, mod_glm.predict()로 계산한 예측치)
    
    $$Pearson residuals = \frac{y - N\hat{p}}{\sqrt{N\hat{p} \left ( 1 - \hat{p} \right)} }$$
    
* 하나하나의 예측 결과의 시행횟수는 당연히 1이되기 때문에 피어슨 잔차는 실질적으로 다음과 같이 계산됨

    $$Pearson residuals = \frac{y - N\hat{p}}{\sqrt{\hat{p} \left ( 1 - \hat{p} \right)} }$$

### 6.4.3 피어슨 잔차의 해석

* 피어슨 잔차의 분모에 있는 $N\hat{p} \left ( 1 - \hat{p} \right)$ 은 이항분포의 분산 값과 일치함. 그 값에 루트를 취한 것이므로 분모는 이항분포의 표준편차로 볼 수 있음

* 정규선형모델에서는 종속변수와 predict() 함수로 구한 예측값의 차이를 잔차로 사용함

     - 피어슨 잔차는 일반 잔차를 분포의 표준편차로 나눈것으로 해석할 수 있음


* 피어슨 잔차의 제곱합은 **피어슨 카이제곱통계량** 이라고도 부르며, 모델 적합도의 지표가 됨

    - 피어슨 잔차를 피어슨 카이제곱통계량의 부호 달린 루트값으로 끌어내는 사람도 있음

### 6.4.4 피어슨 잔차(실습)

In [5]:
# 예측한 성공확률
pred = mod_glm.predict()

# 종속변수(시험 합격여부)
y = test_result.result

# 피어슨 잔차
pearson_resid = (y - pred) / np.sqrt(pred * (1 - pred))
pearson_resid.head(6)

0   -0.102351
1   -0.102351
2   -0.102351
3   -0.102351
4   -0.102351
5   -0.102351
Name: result, dtype: float64

In [6]:
# 모델에서 직접 얻을 수도 있음(피어슨 잔차)

mod_glm.resid_pearson.head(6)

0   -0.102351
1   -0.102351
2   -0.102351
3   -0.102351
4   -0.102351
5   -0.102351
dtype: float64

* **피어슨 잔차제곱합은 피어슨 카이제곱통계량이 됨**

In [7]:
np.sum(mod_glm.resid_pearson ** 2)

84.911

* **이 결과는 summary() 함수에도 출력되며, 아래와 같이 꺼낼 수 있음**

In [8]:
mod_glm.pearson_chi2

84.911

* **피어슨 카이제곱통계량에 대한 P-value 산출**

In [9]:
# hours 적합 모델
1 - stats.chi2.cdf(84.9 , 98)

0.825

In [10]:
# Null 모델
1 - stats.chi2.cdf(137.99 , 98)

0.005

* hours 적합 모델의 피어슨 카이제곱 통계량에 대한 P-value가 0.05 보다 크므로 '현 모델 ```result ~ hours```이 참이다'라는 귀무가설을 기각할 수 없음(Residual Device)

* Null 모델의 피어슨 카이제곱 통계량에 대한 P-value가 0.05 보다 작으므로 'Null 모델이 참이다'의 귀무가설이 기각되며 Null 모델은 적합결여가 됨(즉, Null 모델은 알맞지 않다)(Null deviance)

### 6.4.5 deviance

* **deviance** 는 모델의 적합도를 평가하는  지표

    * deviance가 크면 모델이 맞지 않는다고 평가할 수 있음

* 로지스틱 회귀의 로그우도 함수를 $logL(\beta_0, \beta_1 ; N, m)$ 이라고 할 때 계수 $\beta_0$, $\beta_1$을 바꿈으로써 우도가 변함

    - 여기서 최대우도법을 추정한 로지스틱 회귀의 계수에 근거한 로그우도를 $logL(\beta_{glm} ; y)$ 라고 하며,
    
    - 모든 합격 여부를 완전히 예측할 수 있을 때의 로그우도를 $logL(\beta_{max};y)$ 라고 하면
    
    - deviance는 다음과 같이 계산할 수 있다. 
    
    $$deviance = 2\left[logL(\beta_{max};y) - logL(\beta_{glm} ; y) \right]$$

### 6.4.6 deviance의 해석

* **deviance**는 잔차제곱합을 우도처럼 표현한 것

    - 즉, 최대우도법의 결과와 deviance라는 손실을 최소화하도록 파라미터를 추정한 결과는 일치함
    
* $logL(\beta_{max};y)$ 는 종속변수를 완전히 예측할 수 있을 때의 로그우도. 즉, 합격(1)이라면 성공확률 100%, 불합격이라면 성공확률 0%로 예측할 때의 로그 우도

    - 이 값보다 로그우도를 높이는 것은 불가능하며, 여기서 차이를 측정한 것이 **deviance** 임

### 6.4.7 deviance와 우도비 검정

* deviance를 계산할 때 로그우도의 차이에 2를 곱함. 2를 곱한 이유는 우도비 검정을 할때 편하기 때문

* deviance는 일반선형모델에서 잔차제곱합과 같은 의미를 갖는 지표

    - 때문에 모델의 deviacne의 차이를 통계량으로 한 검정은 분산분석처럼 해석이 가능함
    
    - 이 때 deviance를 앞서와 같이 정의해두면 deviance 차이가 카이제곱분포로 불리는 분포에 점근적으로 따르게 된다는 것이 증명됨


* **deviance의 차이를 검정하는 것** 을 **우도비 검정**이라고 함

    - R언어에서는 분산분석과 우도비 검정이 같은 anova 함수로 구현됨

### 6.4.8 deviance 잔차


* 이항분포에선 **deviance 잔차** 는 deviance 잔차제곱합이 deviance가 된다는 사실로 계산함

* deviance 잔차를 계산하는 코드는 아래와 같음

In [11]:
# 예측한 성공확률
pred = mod_glm.predict()

# 종속변수(테스트 합격여부)
y = test_result.result

# 합격여부를 완전히 예측할 수 있을 때의 로그우도와 잔차
resid_tmp = 0 - np.log(sp.stats.binom.pmf(k = y, n = 1, p = pred))
                       
# deviance 잔차
deviance_resid = np.sqrt(2 * resid_tmp)* np.sign(y - pred)

# 결과 확인
deviance_resid.head(3)

0   -0.144369
1   -0.144369
2   -0.144369
Name: result, dtype: float64

* deviance 잔차는 모델에서 직접 얻을 수도 있음

In [12]:
mod_glm.resid_deviance.head(3)

0   -0.144369
1   -0.144369
2   -0.144369
dtype: float64

* **deviance 잔차제곱합**은 **deviance**가 됨 (이 값은 summary 함수에도 출력됨)

In [13]:
np.sum(mod_glm.resid_deviance ** 2)

68.028

In [14]:
mod_glm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                 result   No. Observations:                  100
Model:                            GLM   Df Residuals:                       98
Model Family:                Binomial   Df Model:                            1
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -34.014
Date:                Sun, 18 Sep 2022   Deviance:                       68.028
Time:                        19:14:00   Pearson chi2:                     84.9
No. Iterations:                     6   Pseudo R-squ. (CS):             0.5032
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -4.5587      0.901     -5.061      0.000      -6.324      -2.793
hours          0.9289      0.174      5.345      0.000       0.588       1.270
==============================================================================
"""

* 결국 deviance는 완전모델(포화모델, saturated model)과의 차이를 나타내며 null deviance 는 독립변수가 없는 null 모델의 이탈도를 나타냄


* 회귀 모델의 성능인 Pseudo R-squ. (CS) 값은 Deviance로부터 도출함 

> **(참고)** https://datascienceschool.net/03%20machine%20learning/10.01%20%EB%A1%9C%EC%A7%80%EC%8A%A4%ED%8B%B1%20%ED%9A%8C%EA%B7%80%EB%B6%84%EC%84%9D.html

### 6.4.9 교차 엔트로피 오차

* 머신러닝에서는 로지스틱 회귀를 **교차 엔트로피 오차**의 최소화 관점으로 설명하는 일이 자주 있음 

* 이항분포의 확률질량함수

   $$Bin \left( m | N, p \right) = _{n}\textrm{C}_{m} \cdot p^m \cdot \left(1-p \right)^{N-m} $$

* 여기서 하나하나의 데이터에 대해 시행횟수 N은 언제나 1이 되므로 다음과 같이 된다. (m은 0이나 1 밖에 되지 않는다.)

   $$Bin \left( m | N, p \right) = p^m \cdot \left(1-p \right)^{1-m} $$

* 분석 사례에 맞추어서 0이나 1의 합격 여부를 $y$, 예측한 합격률을 $\hat{p}$ 라고 함 

   $$Bin \left( y | 1, \hat{p} \right) = \hat{p}^{y} \cdot \left( 1- \hat{p} \right)^{1-y} $$

* 샘플 사이즈 T의 우도함수

   $$\prod_{i=1}^{T} \hat{p_i}^{y_i} \cdot \left( 1- \hat{p_i} \right)^{1-y_i} $$

* 우도 함수에 -1을 곱하면 아래와 같으며 이를 **엔트로피 오차**라고 함

$$ -\sum_{i=1}^{T} \left [ y_i log\hat{p_i} + \left ( 1-y_i \right ) log\left ( 1-\hat{p_i} \right )\right ]$$

* 모집단 분포가 이항분포라고 가정한 경우 deviance와 같은 의미를가짐 

* 교차 엔트로피 오차를 최소로 하는 것은 deviance를 최소로 하는 것과 동일하며, 로지스틱 회귀의 로그우도를 최대로 하는 것과 같음